Colab에서 R 사용 가능

[링크 텍스트](https://www.youtube.com/watch?v=ugNlbyOlWlQ) #이 분 없었으면 평생 헤맸을듯

In [1]:
#엑셀 불러올 준비
system("gdown --id 1q0JuMeYNscfq2BrGuEfk-_fDZTj2y5k-")
library(readxl)

In [2]:
df_master = read_excel("df_master.xlsx") #구글 드라이브에서 주소 가져올 때 링크가 있는 모든 사용자에게 공개 꼭! 해줘야함 

In [ ]:
unique(df_master$CLAC1_NM)

[1] "건강식품" "냉동식품" "냉장식품" "과일"     "축산물"

In [ ]:
length(unique(df_master$CLNT_ID)) # 고유 ID는 9만 33397명

[1] 93397

In [ ]:
#library(haven)
#df_master$MONTH <- format(as.Date(df_master$SESS_DT), "%m")
#df_master$MONTH <- as.integer(df_master$MONTH)

In [ ]:
#pdf_master <- pdata.frame(df_master, index=c("CLNT_ID","MONTH"))

#CLNT_ID와 함께 시간번수를 index로 넣으려고 했으나 오류 발생 
#아마도 SESS_DT가 POSIX-format(날짜형식)이라서 그런 것 같음. 

Warning message in pdata.frame(df_master, index = c("CLNT_ID", "MONTH")):
“duplicate couples (id-time) in resulting pdata.frame
 to find out which, use, e.g., table(index(your_pdataframe), useNA = "ifany")”


In [4]:
install.packages("plm")
library(plm)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘miscTools’, ‘rbibutils’, ‘bdsmatrix’, ‘zoo’, ‘sandwich’, ‘lmtest’, ‘maxLik’, ‘Rdpack’, ‘Formula’




In [ ]:
#SESS_DT 형식으로 인해 index에 추가하지 못하는 오류 
# 참고: https://stackoverflow.com/questions/31773723/r-plm-regression-with-time-in-posix-format
#install.packages('plm')
library(dplyr)
df_master <- df_master %>%
    group_by(CLNT_ID) %>%
    mutate(clock = seq_along(CLNT_ID))
panel <- pdata.frame(df_master, c("CLNT_ID", "SESS_DT"))


Attaching package: ‘plm’


The following objects are masked from ‘package:dplyr’:

    between, lag, lead


Warning message in pdata.frame(df_master, c("CLNT_ID", "SESS_DT")):
“duplicate couples (id-time) in resulting pdata.frame
 to find out which, use, e.g., table(index(your_pdataframe), useNA = "ifany")”


In [ ]:
df_master_health <- subset(df_master, CLAC1_NM=="건강식품") 
df_master_ice <- subset(df_master, CLAC1_NM=="냉동식품") 
df_master_ref <- subset(df_master, CLAC1_NM=="냉장식품") 
df_master_fruit <- subset(df_master, CLAC1_NM=="과일")
df_master_meet  <- subset(df_master, CLAC1_NM=="축산물")
#따로 분석해볼까 싶어서 분리

In [ ]:
panel <- pdata.frame(df_master, c("CLNT_ID", "SESS_DT"))

Warning message in pdata.frame(df_master, c("CLNT_ID", "SESS_DT")):
“duplicate couples (id-time) in resulting pdata.frame
 to find out which, use, e.g., table(index(your_pdataframe), useNA = "ifany")”


In [13]:
#pooltest 함수를 통해 고정효과 모델에 대한 검정
pooltest(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, index=c("CLNT_ID"), data=df_master, model="within")

ERROR: ignored

In [21]:
a <- plm(log(PD_AMOUNT) ~ TEMP_AVG+RAINFALL_DAY+DUST, index=c("CLNT_ID"), data=df_master, model="within")
summary(a)

Oneway (individual) effect Within Model

Call:
plm(formula = log(PD_AMOUNT) ~ TEMP_AVG + RAINFALL_DAY + DUST, 
    data = df_master, model = "within", index = c("CLNT_ID"))

Unbalanced Panel: n = 93397, T = 1-576, N = 188189

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-5.703677 -0.064606  0.000000  0.051984  3.869823 

Coefficients:
                Estimate  Std. Error t-value  Pr(>|t|)    
TEMP_AVG     -0.00329662  0.00049844 -6.6139 3.763e-11 ***
RAINFALL_DAY  0.00041124  0.00015254  2.6959  0.007021 ** 
DUST          0.00011689  0.00018569  0.6295  0.529010    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    30702
Residual Sum of Squares: 30684
R-Squared:      0.00060797
Adj. R-Squared: -0.98413
F-statistic: 19.2212 on 3 and 94789 DF, p-value: 1.8704e-12

In [12]:
# 확률효과를 검정하기 위한 LM 검정 통계량
plmtest(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, index=c("CLNT_ID"), data=df_master, model="random")

#확률효과 검정 결과에 의하면 1%유의 수준에서 확률효과가 없다는 귀무가설을 기각하게 되어 패널 자료에 통계적으로 유의미한 확률효과가 있다고 볼 수 있다. 


	Lagrange Multiplier Test - (Honda) for unbalanced panels

data:  log(PD_AMOUNT) ~ factor(SEASON) + TEMP_AVG + RAINFALL_DAY + HUMID_AVG +  ...
normal = 1121.5, p-value < 2.2e-16
alternative hypothesis: significant effects


In [6]:
b <- plm(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, index=c("CLNT_ID"), data=df_master, model="random")
summary(b)
#와씨 r제곱 0.86

Oneway (individual) effect Random Effect Model 
   (Swamy-Arora's transformation)

Call:
plm(formula = log(PD_AMOUNT) ~ factor(SEASON) + TEMP_AVG + RAINFALL_DAY + 
    HUMID_AVG + DUST, data = df_master, model = "random", index = c("CLNT_ID"))

Unbalanced Panel: n = 93397, T = 1-576, N = 188189

Effects:
                 var std.dev share
idiosyncratic 0.3230  0.5683 0.278
individual    0.8405  0.9168 0.722
theta:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4731  0.4731  0.5985  0.6314  0.7039  0.9742 

Residuals:
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-5.2624 -0.3469  0.0050 -0.0085  0.3333  4.0646 

Coefficients:
                        Estimate  Std. Error  z-value  Pr(>|z|)    
(Intercept)          10.96129962  0.01852359 591.7482 < 2.2e-16 ***
factor(SEASON)spring -0.36336578  0.00822108 -44.1993 < 2.2e-16 ***
factor(SEASON)summer -0.38379421  0.00708165 -54.1956 < 2.2e-16 ***
TEMP_AVG             -0.00468985  0.00057838  -8.1086 5.121e-16 ***
RAINFALL_DAY       

In [8]:
phtest(a, b)


	Hausman Test

data:  log(PD_AMOUNT) ~ factor(SEASON) + TEMP_AVG + RAINFALL_DAY + HUMID_AVG +  ...
chisq = 570.75, df = 6, p-value < 2.2e-16
alternative hypothesis: one model is inconsistent


### Hausman Test 해석

카이제곱 통계량은 570.75으로 1% 유의수준에서 고정효과모형과 확률효과모형이 동일하다는 귀무가설을 기각한다. 따라서 본 연구에서 사용한 데이터세트의 경우 확률효과모형이 고정효과모형에 비해 설명력이 더 높은 타당한 모형이라 판단하였다.

In [15]:
c <- plm(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, index=c("CLNT_ID"), data=df_master, model="between", effect = "twoways")
summary(c)

ERROR: ignored

In [17]:
d <- plm(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, index=c("CLNT_ID"), data=df_master, model="within", effect = "twoways")
summary(d)

# 패널 회귀분석 공부좀 해야됨 
## 고정효과, 임의효과 개념 아직도 제대로 모름ㅠ 

